In [1]:
import import_ipynb
import pandas as pd
import scrapy
import scrapy.crawler as crawler
from scrapy.crawler import CrawlerProcess, CrawlerRunner
from multiprocessing import Process, Queue
from twisted.internet import reactor
import html2text
import json
import rules
from rules import *
import find_triggers
from find_triggers import *


#define URLs
urls = [
        'https://aajtak.intoday.in/'
        ]
#define max sites
max_sites = 1

#define what you want the JSON file of the website content to be
name_of_content_file = 'test_aajtak'

importing Jupyter notebook from rules.ipynb
importing Jupyter notebook from initialise_content_and_finder.ipynb
importing Jupyter notebook from find_triggers.ipynb


In [2]:
#don't touch this stuff
text = []
site_count = 0
class QuotesSpider(scrapy.Spider):
    global max_sites
    name = "quotes"
    
    custom_settings = {
        'CLOSESPIDER_ITEMCOUNT': max_sites,
        'LOG_ENABLED': False
    }
    
    start_urls = urls
    
    def parse(self, response):
        global site_count
        global max_sites
        if (site_count < max_sites):
            page = response.url.split("/")[-2]
            text.append(response)
            site_count += 1
            yield from response.follow_all(css='a', callback=self.parse) 

#run script
#Need to restart script each time it is run
runner = CrawlerRunner()
d = runner.crawl(QuotesSpider)
d.addBoth(lambda _: reactor.stop())
reactor.run()

h = html2text.HTML2Text()

def decode(body):
    return h.handle(str(body.body.decode('utf-8')))

content = [decode(page) for page in text]
content = [con.replace("\n\n", " ") for con in content]
content = [con.replace("\n", " ") for con in content]

pages_content = {}
counter = 0
for url in QuotesSpider.start_urls:
    pages_content.update({url : content[counter]})
    counter +=1
    
import json
# Serialize data into file:
with open(name_of_content_file + ".json", 'w') as file:
    json.dump(pages_content, file)

In [3]:
pages_content

{'https://aajtak.intoday.in/': '[![Tez Aajtak](https://smedia2.intoday.in/aajtak/resources/v5_theme/components/images/defaultimg-130x72.jpg?size=138:74)](http://www.tezaajtak.in/) [![Indiatoday Hindi](https://smedia2.intoday.in/aajtak/resources/v5_theme/components/images/defaultimg-130x72.jpg?size=138:74)](http://www.indiatodayhindi.in/)   * ###  [मूवी मसाला  ](/moviemasala "मूवी मसाला") [फिल्\u200dमी खबरें](/news-on-films.html "फिल्\u200dमी खबरेंें")[ट्रेलर](/new-trailer- and-new-songs-of-upcoming-hindi-films-of-bollywood.html "ट्रेलरें")[गॉसिप](/filmy-gossips.html "गॉसिपें")[फिल्म समीक्षा](/film- review.html "फिल्म समीक्षाें")[इंटरव्यू](/celebrity-interview.html "इंटरव्यूें")[छोटा पर्दा ](/small-screen.html "छोटा पर्दा ें")[स्टार टॉक ](/karyakrams/video/movie-masala.html "स्टार टॉक ें")[वायरल वीडियो ](/viral- video.html "वायरल वीडियो ें")[बर्थडे स्पेशल ](/celebrity-birthday-today.html "बर्थडे स्पेशल ें")[परदेसी सिनेमा](/photos/hollywood.html "परदेसी सिनेमाें")   * ###  [लाइफस्टाइल  ]

In [9]:


#import content (for the moment)

#define terms lists

risky = ['पोंटिंग की दो टूक- अश्विन को इस बार ‘मांकड़िंग’ नहीं करने दूंगा']

#Define rules to be run
rule_2_to_run = rule_2(pages_content, high_risk_words = risky)

#Get results from running the rules
rule_2_results = find_triggered_items(rule_to_run = rule_2_to_run, content = pages_content)

rule_2_results

#phrase
#spacy models


,phrase,high_risk_words_in,rule,url
0,[![Tez Aajtak](https://smedia2.intoday.in/aajt...,पोंटिंग की दो टूक- अश्विन को इस बार ‘मांकड़िंग...,Rule 2: High-risk words in sentences,https://aajtak.intoday.in/
1,\(PTI Photo\)](https://smedia2.intoday.in/aajt...,पोंटिंग की दो टूक- अश्विन को इस बार ‘मांकड़िंग...,Rule 2: High-risk words in sentences,https://aajtak.intoday.in/


In [ ]:
#import content (for the moment)
import json
pages_content = json.load(open("RM_scrape_20200819.json"))

#define terms lists
sign_up_language = ['Join']
offers = ['available', 'complex and specific risks']
risky = ['suit your risk appetite']
exclude = []
disclaimer_sign_up = ['By submitting this form, you consent to being contacted by a Red Marker sales representative and receiving promotional materials (including e-newsletters) from Red Marker from time to time.']
disclaimer_legal = ['This is a legal disclaimer.']

#Define rules to be run
rule_1_to_run = rule_1(pages_content, offer_lan = offers, high_risk_words = risky, exclusions = exclude)
rule_2_to_run = rule_2(pages_content, high_risk_words = risky)
rule_3_to_run_sign_up = rule_3(pages_content, offer_lan = sign_up_language, disclaimers = disclaimer_sign_up)
rule_4_to_run_legal = rule_4(pages_content, disclaimers = disclaimer_legal)

#Get results from running the rules
rule_3_results = find_triggered_items(rule_to_run = rule_3_to_run_sign_up, content = pages_content)
rule_2_results = find_triggered_items(rule_to_run = rule_2_to_run, content = pages_content)
rule_1_results = find_triggered_items(rule_to_run = rule_1_to_run, content = pages_content)
rule_4_results = find_triggered_items(rule_to_run = rule_4_to_run_legal, content = pages_content)

risky_promotions = pd.concat([rule_1_results, rule_2_results, rule_3_results, rule_4_results])
risky_promotions